In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B2,preprocess_input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger

2025-04-14 09:36:40.533684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744623400.733739      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744623400.793539      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

I0000 00:00:1744623432.015916      31 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1744623432.019069      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [5]:
x_train=tf.keras.applications.efficientnet.preprocess_input(x_train)
x_test=tf.keras.applications.efficientnet.preprocess_input(x_test)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [7]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [8]:
base_model=EfficientNetV2B2(
    weights='imagenet',
    include_top=False, 
    input_shape=(64,64,3) 
)

35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [9]:
base_model.trainable=False

In [10]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-b2 (Functional)       │ (None, 2, 2, 1408)          │       8,769,374 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1408)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,442,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,901,186 (41.58 MB)

 Trainable params: 2,128,228 (8.12 MB)

 Non-trainable params: 8,772,958 (33.47 MB)

In [11]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [12]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [13]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [14]:
import time
start_time=time.time()

In [15]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [16]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 96s 73ms/step - accuracy: 0.0495 - loss: 4.8511 - val_accuracy: 0.3511 - val_loss: 2.7007
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 64s 49ms/step - accuracy: 0.2016 - loss: 3.4142 - val_accuracy: 0.4271 - val_loss: 2.2467
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 57s 48ms/step - accuracy: 0.2703 - loss: 2.9845 - val_accuracy: 0.4607 - val_loss: 2.0649
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 58s 49ms/step - accuracy: 0.3057 - loss: 2.7696 - val_accuracy: 0.4806 - val_loss: 1.9554
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 57s 49ms/step - accuracy: 0.3324 - loss: 2.6426 - val_accuracy: 0.4950 - val_loss: 1.8872
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 56s 48ms/step - accuracy: 0.3516 - loss: 2.5382 - val_accuracy: 0.5048 - val_loss: 1.8432
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 55s 47ms/step - accuracy: 0.3641 - loss: 2.4700 - val_accuracy: 0.5154 - val_loss: 1.7967
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 55s 47ms/step - accuracy: 0.3760 -

In [17]:
head_training_time=time.time()-start_time

In [18]:
base_model.trainable=True

In [19]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [20]:
start_fine_tune=time.time()

In [21]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


E0414 09:57:32.155669      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:57:32.335942      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:57:32.664166      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:57:32.849253      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:57:33.287605      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:57:33.511367      31 gpu_timer.cc:82] Del

  69/1172 ━━━━━━━━━━━━━━━━━━━━ 7:45 422ms/step - accuracy: 0.1411 - loss: 4.0765

E0414 09:58:34.936729      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:58:35.118110      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:58:35.433382      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:58:35.619711      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:58:36.063513      31 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0414 09:58:36.286087      31 gpu_timer.cc:82] Del

1172/1172 ━━━━━━━━━━━━━━━━━━━━ 190s 134ms/step - accuracy: 0.1767 - loss: 3.8208 - val_accuracy: 0.3420 - val_loss: 2.6998
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 110s 73ms/step - accuracy: 0.2702 - loss: 3.0615 - val_accuracy: 0.4178 - val_loss: 2.2461
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 85s 72ms/step - accuracy: 0.3331 - loss: 2.6978 - val_accuracy: 0.4626 - val_loss: 2.0249
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 84s 72ms/step - accuracy: 0.3654 - loss: 2.4947 - val_accuracy: 0.5003 - val_loss: 1.8543
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 86s 73ms/step - accuracy: 0.3933 - loss: 2.3369 - val_accuracy: 0.5255 - val_loss: 1.7390


In [22]:
fine_tune_time=time.time()-start_fine_tune

In [23]:
test_loss,test_accuracy=model.evaluate(x_test,y_test,verbose=1)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.3402 - loss: 2.6852

Test Accuracy: 34.25%
Total training time: 1735.44 seconds


In [24]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3437 - loss: 2.6995
Validation accuracy: 34.2000%


In [25]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step


In [26]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for EfficientNetV2B2:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for EfficientNetV2B2:
Accuracy: 0.3425
Precision: 0.3388
Recall: 0.3425
F1-Score: 0.3357


In [27]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["EfficientNetV2B2"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [28]:
print(results)

              Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  EfficientNetV2B2         0.3425                0.342   2.680446   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0              1178.45297            556.986347      10901186  


In [29]:
results.to_csv("EfficientNetV2B2.csv",index=False)
print("\nResults saved to EfficientNetV2B2.csv")


Results saved to EfficientNetV2B2.csv
